In [ ]:
!pip install wandb

In [3]:
import torch, torchvision, wandb

🔸wandb config

In [ ]:
configs = {
            "learning_rate": 0.001,
            "epochs": 20,
            "batch_size": 64,
           }

wandb.init(project="Fashion-MNIST-by-Torch", config=configs)
config = wandb.config


🔸Class

In [144]:
class mnist_classifire(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.L1=torch.nn.Linear(784,256)
        self.L2=torch.nn.Linear(256,64)
        self.L3=torch.nn.Linear(64,10)
        # self.L4=torch.nn.Linear(128,10)


    def forward(self,x):
        x = x.reshape((x.shape[0],784))

        z = self.L1(x)
        z = torch.relu(z)
        z = torch.dropout(z,0.2,train = True)

        z = self.L2(z)
        z = torch.relu(z)
        z = torch.dropout(z,0.2,train = True)
        z = self.L3(z)
        z = torch.relu(z)
        z = torch.dropout(z,0.2,train = True)

        y = torch.softmax(z,dim = 1)
        return y


In [145]:
device=torch.device("cuda")
model=mnist_classifire()
model=model.to(device)
model.train(True)

mnist_classifire(
  (L1): Linear(in_features=784, out_features=256, bias=True)
  (L2): Linear(in_features=256, out_features=64, bias=True)
  (L3): Linear(in_features=64, out_features=10, bias=True)
)

In [146]:
batch=64
epoch=20
lr=0.001

In [147]:
data_transform=torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0),(1))]
)
dataset = torchvision.datasets.FashionMNIST("./dataset",train=True,download=True,transform=data_transform)

In [148]:
train_data = torch.utils.data.DataLoader(dataset,batch_size=batch,shuffle=True)

In [149]:
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
loss_func = torch.nn.CrossEntropyLoss()

In [150]:
def cal_acc(y_hat,labels):
    _,y_hat_max=torch.max(y_hat,1)
    acc=torch.sum(y_hat_max==labels.data,dtype=torch.float64)/len(y_hat)
    return acc


🔸Train

In [151]:
for ep in range(epoch):
    train_loss = 0.0
    train_acc = 0.0

    for im,labels in train_data:
        im = im.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        #forwarding
        y_hat = model(im)

        #backwarding
        loss = loss_func(y_hat,labels)
        loss.backward()

        #update
        optimizer.step()

        train_loss += loss
        train_acc += cal_acc(y_hat,labels)

    total_loss  =  train_loss/len(train_data)
    total_acc  =  train_acc/len(train_data)

    print(f"epoch:{ep} , Loss:{total_loss} , accuracy: {total_acc}")

    wandb.log({'epochs': ep,
      'train_acc': total_acc,
      'train_loss': total_loss})

epoch:0 , Loss:1.8520758152008057 , accuracy: 0.609391657782516
epoch:1 , Loss:1.7961249351501465 , accuracy: 0.6577991737739872
epoch:2 , Loss:1.785601019859314 , accuracy: 0.6678271588486141
epoch:3 , Loss:1.781274676322937 , accuracy: 0.6716584488272921
epoch:4 , Loss:1.781451940536499 , accuracy: 0.6718416844349681
epoch:5 , Loss:1.77535080909729 , accuracy: 0.6779217750533049
epoch:6 , Loss:1.7772291898727417 , accuracy: 0.6750233208955224
epoch:7 , Loss:1.7766743898391724 , accuracy: 0.6744069829424307
epoch:8 , Loss:1.7744252681732178 , accuracy: 0.6761727078891258
epoch:9 , Loss:1.7696712017059326 , accuracy: 0.681586487206823
epoch:10 , Loss:1.7722129821777344 , accuracy: 0.6781216684434968
epoch:11 , Loss:1.77056086063385 , accuracy: 0.6800872867803838
epoch:12 , Loss:1.7701897621154785 , accuracy: 0.6784215085287847
epoch:13 , Loss:1.7701143026351929 , accuracy: 0.6805037313432836
epoch:14 , Loss:1.7665692567825317 , accuracy: 0.6826525852878464
epoch:15 , Loss:1.76651799678

🔸Save weights

In [143]:
torch.save(model.state_dict(), "fashion-mnist.pth")

In [153]:
import cv2
import numpy as np

🔸Inference

In [155]:
model.eval()

img=cv2.imread("shirt.png")
img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.resize(img,(28,28))
tensor=data_transform(img).unsqueeze(0).to(device)

y_hat=model(tensor)

y_hat=y_hat.cpu().detach().numpy()
output=np.argmax(y_hat)
output

2